In [2]:
import pandas as pd
import requests
import re
import os

# **Use #reviewkhachsan193.csv to get required column**

In [3]:
df = pd.read_csv("/content/drive/MyDrive/pizza/CÔNG NGHỆ ĐỔI MỚI (1)/CUỐI KỲ/UP file csv lên/#reviewkhachsan193.csv")

In [4]:
df.head(2)

,authorMeta/digg,authorMeta/fans,authorMeta/heart,webVideoUrl,isAd,isSponsored,playCount,diggCount,commentCount,shareCount,videoMeta/duration,videoMeta/subtitleLinks/1/downloadLink,text
0,42,33,2553,https://www.tiktok.com/@coinecoinee/video/7502...,False,False,907,42,2,5,132,https://api.apify.com/v2/key-value-stores/Qivr...,Review khách sạn view triệu đô giữa lòng Bangk...
1,1969,11500,215000,https://www.tiktok.com/@tony_khoa/video/740338...,False,False,10300,36,3,35,60,https://api.apify.com/v2/key-value-stores/Qivr...,Midas hotel khách sạn đẹp giữa trung tâm phú n...


In [5]:
df.columns.to_list()

['authorMeta/digg',
 'authorMeta/fans',
 'authorMeta/heart',
 'webVideoUrl',
 'isAd',
 'isSponsored',
 'playCount',
 'diggCount',
 'commentCount',
 'shareCount',
 'videoMeta/duration',
 'videoMeta/subtitleLinks/1/downloadLink',
 'text']

# **Process all files**



In [6]:
# Đường dẫn tới thư mục chứa các tệp CSV
folder_path = '/content/drive/MyDrive/pizza/CÔNG NGHỆ ĐỔI MỚI (1)/CUỐI KỲ/UP file csv lên/excel/'

# Danh sách các cột cần giữ lại
columns_to_keep = [
    'authorMeta/digg',
    'authorMeta/fans',
    'authorMeta/heart',
    'webVideoUrl',
    'isAd',
    'isSponsored',
    'playCount',
    'diggCount',
    'commentCount',
    'shareCount',
    'videoMeta/duration',
    'videoMeta/subtitleLinks/1/downloadLink',
    'text'
]

# Lấy danh sách tất cả các tệp CSV trong thư mục
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Nếu có tệp CSV hợp lệ trong thư mục
if csv_files:
    # Danh sách lưu trữ các DataFrame đã xử lý
    processed_data = []

    # Duyệt qua tất cả các tệp CSV
    for csv_file in csv_files:
        file_path = os.path.join(folder_path, csv_file)

        # Đọc tệp CSV vào DataFrame
        df = pd.read_csv(file_path)

        # Kiểm tra nếu cột videoMeta/subtitleLinks/1/downloadLink có trong DataFrame
        if 'videoMeta/subtitleLinks/1/downloadLink' in df.columns:
            # Chỉ giữ lại các cột yêu cầu
            df = df[columns_to_keep]

            # Lọc các dòng không có giá trị null ở cột videoMeta/subtitleLinks/1/downloadLink
            df = df[df['videoMeta/subtitleLinks/1/downloadLink'].notnull()]

            # Sắp xếp các cột theo đúng thứ tự yêu cầu
            df = df[columns_to_keep]

            # In tên tệp đã đọc
            print(f"Đã đọc tệp: {csv_file}")

            # Thêm DataFrame đã xử lý vào danh sách
            processed_data.append(df)

    # Nếu có dữ liệu đã được xử lý, kết hợp tất cả vào một DataFrame duy nhất
    if processed_data:
        final_df = pd.concat(processed_data, ignore_index=True)

        # Hiển thị một số dòng đầu tiên của kết quả
        print(final_df.head(2))

        # Nếu muốn lưu kết quả vào một tệp CSV mới
        # final_df.to_csv('/content/drive/MyDrive/output_file.csv', index=False)
    else:
        print("Không có dữ liệu hợp lệ nào để xử lý.")
else:
    print("Không tìm thấy tệp CSV hợp lệ trong thư mục.")


Đã đọc tệp: reviewhotel189.csv
Đã đọc tệp: reviewhotel209.csv
Đã đọc tệp: hotelsaigon190.csv
Đã đọc tệp: hotelhanoi234.csv
Đã đọc tệp: hotelhalong152.csv
Đã đọc tệp: hotelvungtau158.csv
Đã đọc tệp: hotelnhatrang262.csv
Đã đọc tệp: hotelluxury207.csv
Đã đọc tệp: hotelquan1-184.csv
Đã đọc tệp: hotelcantho210.csv
Đã đọc tệp: hoteldalat265.csv
Đã đọc tệp: hotelgiare209.csv
Đã đọc tệp: hotelphuquy170.csv
Đã đọc tệp: hotelphuquoc234.csv
Đã đọc tệp: hoteldanang159.csv
Đã đọc tệp: hotelhue234.csv
Đã đọc tệp: hoteldaklak100.csv
Đã đọc tệp: hotelhoian234.csv
Đã đọc tệp: hotelquangbinh238.csv
Đã đọc tệp: hotelninhthuan260.csv
Đã đọc tệp: hotelhaiphong214.csv
Đã đọc tệp: hotelhagiang230.csv
Đã đọc tệp: hoteltuyenquang210.csv
Đã đọc tệp: #khachsan180lastest.csv
Đã đọc tệp: review_resort3.csv
Đã đọc tệp: homestay_oldest.csv
Đã đọc tệp: review_resort2.csv
Đã đọc tệp: reviewhomestay_latest.csv
Đã đọc tệp: homestay_latest.csv
Đã đọc tệp: #khachsan162.csv
Đã đọc tệp: review_resort1.csv
Đã đọc tệp: revie

In [7]:
final_df = final_df.drop_duplicates(subset='webVideoUrl', keep='first')

In [8]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1281 entries, 0 to 2305
Data columns (total 13 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   authorMeta/digg                         1281 non-null   int64 
 1   authorMeta/fans                         1281 non-null   int64 
 2   authorMeta/heart                        1281 non-null   int64 
 3   webVideoUrl                             1281 non-null   object
 4   isAd                                    1281 non-null   bool  
 5   isSponsored                             1281 non-null   bool  
 6   playCount                               1281 non-null   int64 
 7   diggCount                               1281 non-null   int64 
 8   commentCount                            1281 non-null   int64 
 9   shareCount                              1281 non-null   int64 
 10  videoMeta/duration                      1281 non-null   int64 
 11  videoMeta

# **Change script link suffix to pattern -vie-VN**

In [9]:
def fix_subtitle_link(url):
    if not url.endswith('-vie-VN'):
        url = url.rsplit('-', 1)[0] + '-vie-VN'
    return url

final_df['videoMeta/subtitleLinks/1/downloadLink'] = final_df['videoMeta/subtitleLinks/1/downloadLink'].apply(fix_subtitle_link)

In [10]:
final_df.insert(0, 'id', ['video' + str(i).zfill(2) for i in range(1, len(final_df) + 1)])


In [11]:
final_df.to_csv("/content/drive/MyDrive/pizza/CÔNG NGHỆ ĐỔI MỚI (1)/CUỐI KỲ/UP file csv lên/#merge_test1281.csv")

# **Extract script video**

In [12]:
def clean_vtt_from_url(url):
    try:
        response = requests.get(url)
        content_type = response.headers.get("Content-Type", "")

        # Handle JSON response with "record-not-found" error
        if "application/json" in content_type:
            try:
                json_data = response.json()
                if json_data.get("error", {}).get("type") == "record-not-found":
                    return "__NOT_FOUND__"
            except Exception:
                pass  # Continue processing if JSON parsing fails

        # Decode and process VTT content
        text = response.content.decode('utf-8-sig', errors='replace')
        lines = text.splitlines()
        cleaned_lines = []

        for line in lines:
            line = line.strip()
            if line == "WEBVTT":
                continue
            if re.match(r'^\d{2}:\d{2}:\d{2}\.\d{3} --> \d{2}:\d{2}:\d{2}\.\d{3}$', line):
                continue
            if line:
                cleaned_lines.append(line)

        return "\n".join(cleaned_lines)

    except Exception as e:
        print(f"Error processing URL {url}: {e}")
        return "__NOT_FOUND__"

# Process final_df to populate script_video column
indexes_to_drop = []

for index, row in final_df.iterrows():
    video_id = row['id']
    subtitle_url = row['videoMeta/subtitleLinks/1/downloadLink']

    cleaned_text = clean_vtt_from_url(subtitle_url)

    # If subtitle is not found, mark row for deletion
    if cleaned_text == "__NOT_FOUND__":
        print(f"No subtitle found for video_id: {video_id}, will drop this row.")
        indexes_to_drop.append(index)
        continue

    # Directly assign cleaned text to script_video column
    final_df.loc[index, 'script_video'] = cleaned_text
    print(f"Processed script for video_id: {video_id}")

# Drop rows with missing subtitles
final_df.drop(indexes_to_drop, inplace=True)
final_df.reset_index(drop=True, inplace=True)

Processed script for video_id: video01
Processed script for video_id: video02
Processed script for video_id: video03
No subtitle found for video_id: video04, will drop this row.
No subtitle found for video_id: video05, will drop this row.
Processed script for video_id: video06
Processed script for video_id: video07
Processed script for video_id: video08
Processed script for video_id: video09
Processed script for video_id: video10
Processed script for video_id: video11
Processed script for video_id: video12
Processed script for video_id: video13
No subtitle found for video_id: video14, will drop this row.
Processed script for video_id: video15
No subtitle found for video_id: video16, will drop this row.
No subtitle found for video_id: video17, will drop this row.
No subtitle found for video_id: video18, will drop this row.
Processed script for video_id: video19
Processed script for video_id: video20
Processed script for video_id: video21
No subtitle found for video_id: video22, will dro

In [13]:
final_df['id'] = ['video' + str(i).zfill(2) for i in range(1, len(final_df) + 1)]

In [14]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1026 entries, 0 to 1025
Data columns (total 15 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   id                                      1026 non-null   object
 1   authorMeta/digg                         1026 non-null   int64 
 2   authorMeta/fans                         1026 non-null   int64 
 3   authorMeta/heart                        1026 non-null   int64 
 4   webVideoUrl                             1026 non-null   object
 5   isAd                                    1026 non-null   bool  
 6   isSponsored                             1026 non-null   bool  
 7   playCount                               1026 non-null   int64 
 8   diggCount                               1026 non-null   int64 
 9   commentCount                            1026 non-null   int64 
 10  shareCount                              1026 non-null   int64 
 11  vide

In [15]:
final_df.to_csv("fulldata1027.csv")

,id,authorMeta/digg,authorMeta/fans,authorMeta/heart,webVideoUrl,isAd,isSponsored,playCount,diggCount,commentCount,shareCount,videoMeta/duration,videoMeta/subtitleLinks/1/downloadLink,text,script_video
0,video01,8861,32300,1000000,https://www.tiktok.com/@dannie.linhdan_/video/...,False,False,1200000,29900,672,4105,70,https://api.apify.com/v2/key-value-stores/cfrJ...,Khach sạn đỉnh nhất Vũng Tàu trong lòng toai😋 ...,tui Xin bình chọn đây là chiếc khách sạn\nđỉnh...
1,video02,14800,169500,5300000,https://www.tiktok.com/@huyentranxx/video/7403...,False,False,2400000,58700,1146,21300,46,https://api.apify.com/v2/key-value-stores/cfrJ...,Khách sạn tui ở trên Đà Lạt nè mng 🌝 #trend #d...,quay Cho mọi người coi cái phòng tui ở ở Đà Lạ...
2,video03,16700,200400,7300000,https://www.tiktok.com/@traanxngg/video/747518...,False,False,120500,3135,58,1487,65,https://api.apify.com/v2/key-value-stores/cfrJ...,Đà Lạt lại có thêm một boutique hotel quá xinh...,mạnh dạn đoán là sẽ có nhiều bạn bất ngờ trước...
3,video04,12900,59000,1600000,https://www.tiktok.com/@_dmylinh/video/7297203...,False,False,898800,14300,616,2985,90,https://api.apify.com/v2/key-value-stores/cfrJ...,1 chiếc khách sạn chuẩn 3 sao mới toanh với gi...,sau 8 tháng ở Đà Lạt\nthì tui đã có Cho mình m...
4,video05,90800,20300,846600,https://www.tiktok.com/@tienthipham/video/7481...,False,False,755700,19000,794,8444,100,https://api.apify.com/v2/key-value-stores/cfrJ...,Khách sạn Vũng Tàu mà tui cực kỳ thích nha mọi...,thân lắm mới chỉ Cho mọi người\nkhách sạn mà g...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,video1022,213,3218,78300,https://www.tiktok.com/@mivandidauthe/video/75...,False,False,3896,245,24,60,229,https://api.apify.com/v2/key-value-stores/Qivr...,Kinh nghiệm đi du lịch Hong Kong tự túc không ...,đi Hồng Kông tự túc mà không biết bắt đầu từ đ...
1022,video1023,72,1246,18000,https://www.tiktok.com/@longhaichannel.resort/...,False,False,11600,242,20,109,19,https://api.apify.com/v2/key-value-stores/Qivr...,Trả lời @John Tran Gần Long Hải Channel Beach ...,gần mực thì đen gần thiên nhiên thì thư giãn\n...
1023,video1024,8420,172100,3300000,https://www.tiktok.com/@soccon113/video/743845...,False,False,208200,4786,79,228,24,https://api.apify.com/v2/key-value-stores/Qivr...,Mường Thanh Gia Lai về đêm...#review #fyp #rev...,và đây là Mường Thanh Gia Lai\ndiện mạo mới về...
1024,video1025,644,25400,313400,https://www.tiktok.com/@goldencampnamdu/video/...,False,False,1400000,13900,486,367,31,https://api.apify.com/v2/key-value-stores/Qivr...,"Từ TP. Hồ Chí Minh đi Quần Đảo Nam Du, Kiên Gi...",đây là xe Phương Trang\nhãng xe\nkhai thác các...
